In [1]:
from sklearn.datasets import load_breast_cancer
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
cancer = load_breast_cancer(as_frame=True)
cancer.frame.sample(5).round(2)

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension,target
476,14.20,20.53,92.41,618.4,0.09,0.11,0.05,0.03,0.15,0.06,...,27.26,112.10,828.5,0.12,0.34,0.25,0.13,0.25,0.08,1
60,10.17,14.88,64.55,311.9,0.11,0.08,0.01,0.01,0.27,0.07,...,17.45,69.86,368.6,0.13,0.10,0.02,0.03,0.36,0.08,1
380,11.27,12.96,73.16,386.3,0.12,0.11,0.08,0.06,0.20,0.07,...,20.53,84.93,476.1,0.16,0.24,0.22,0.13,0.33,0.09,1
405,10.94,18.59,70.39,370.0,0.10,0.07,0.05,0.03,0.15,0.07,...,25.58,82.76,472.4,0.14,0.16,0.14,0.08,0.23,0.08,1
89,14.64,15.24,95.77,651.9,0.11,0.13,0.10,0.07,0.21,0.06,...,18.24,109.40,803.6,0.13,0.31,0.26,0.14,0.32,0.08,1


In [3]:
특성별_정상필터 = {}

for 특성명 in cancer.data.columns:
    특성열 = cancer.data[특성명]
    Q1, Q3 = np.percentile(특성열, [25, 75])
    IQR = Q3 - Q1
    이상치하한 = Q1 - IQR * 1.5
    이상치상한 = Q3 + IQR * 1.5
    특성별_정상필터[특성명] = 특성열.between(이상치하한, 이상치상한)

In [4]:
정상필터 = np.logical_and.reduce(list(특성별_정상필터.values()))
print(f'전체 관측치 수: {len(정상필터):,}, 정상범위 : {np.sum(정상필터):,}')
cancer.cleaned = cancer.frame.copy()[정상필터]
cancer.cleaned.describe().round(2)

전체 관측치 수: 569, 정상범위 : 398


,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension,target
count,398.00,398.00,398.00,398.00,398.00,398.00,398.00,398.00,398.00,398.00,...,398.00,398.00,398.00,398.00,398.00,398.00,398.00,398.00,398.00,398.00
mean,13.38,18.51,86.40,570.55,0.09,0.09,0.06,0.04,0.17,0.06,...,24.72,98.64,728.47,0.13,0.21,0.21,0.10,0.28,0.08,0.75
std,2.48,3.91,16.84,222.88,0.01,0.03,0.05,0.03,0.02,0.01,...,5.64,22.13,329.65,0.02,0.11,0.15,0.05,0.04,0.01,0.43
min,6.98,9.71,43.79,143.50,0.06,0.02,0.00,0.00,0.12,0.05,...,12.02,50.41,185.20,0.08,0.03,0.00,0.00,0.18,0.06,0.00
25%,11.71,15.70,75.06,420.35,0.08,0.06,0.03,0.02,0.16,0.06,...,20.39,83.92,516.02,0.11,0.14,0.10,0.06,0.25,0.07,1.00
50%,13.00,18.18,84.06,518.30,0.09,0.08,0.05,0.03,0.17,0.06,...,24.58,93.90,632.50,0.13,0.19,0.18,0.08,0.28,0.08,1.00
75%,14.64,20.76,94.73,664.20,0.10,0.11,0.08,0.05,0.19,0.06,...,28.44,109.78,830.58,0.14,0.27,0.30,0.13,0.31,0.09,1.00
max,20.59,29.81,137.80,1320.00,0.13,0.20,0.25,0.13,0.25,0.08,...,40.54,166.40,1872.00,0.19,0.61,0.77,0.25,0.41,0.12,1.00


In [5]:
import joblib

joblib.dump(cancer, 'cancer.joblib')

cancer = joblib.load('cancer.joblib')
print(cancer.keys())
print(cancer.frame.shape, cancer.cleaned.shape)

dict_keys(['data', 'target', 'frame', 'target_names', 'DESCR', 'feature_names', 'filename', 'data_module', 'cleaned'])
(569, 31) (398, 31)


In [11]:
from sklearn.preprocessing import KBinsDiscretizer

구간수 = 4

Xs = pd.DataFrame(np.random.rand(12, 1))
Xs.columns = ['x1']

구간분할기 = KBinsDiscretizer(n_bins=구간수, encode='ordinal', strategy= 'uniform')
구간 = 구간분할기.fit_transform(Xs)
구간표_uniform = pd.DataFrame(구간.astype('int'), columns=['uniform'])
구간분할기 = KBinsDiscretizer(n_bins=구간수, encode='ordinal', strategy='quantile', quantile_method='linear')
구간 = 구간분할기.fit_transform(Xs)
구간표_quantile = pd.DataFrame(구간.astype('int'), columns=['quantile'])
구간분할기 = KBinsDiscretizer(n_bins=구간수, encode='onehot-dense', strategy='quantile', quantile_method='linear')
구간_원핫 = pd.DataFrame(구간분할기.fit_transform(Xs).astype('int'))

pd.concat([Xs, 구간표_uniform, 구간표_quantile, 구간_원핫], axis = 1).sort_values('x1').round(2)

,x1,uniform,quantile,0,1,2,3
11,0.06,0,0,1,0,0,0
0,0.12,0,0,1,0,0,0
6,0.24,0,0,1,0,0,0
4,0.28,1,1,0,1,0,0
1,0.29,1,1,0,1,0,0
8,0.54,2,1,0,1,0,0
5,0.67,2,2,0,0,1,0
2,0.69,2,2,0,0,1,0
10,0.73,3,2,0,0,1,0
9,0.78,3,3,0,0,0,1
